In [12]:
# Auto reloads modules when they change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from src.dataset import wheatDataset
import pandas as pd
import numpy as np

In [14]:
csv_dir = "./data/global-wheat-detection/train.csv"
image_dir = "./data/global-wheat-detection/train"
image_size = 512

df = pd.read_csv(csv_dir)
df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [15]:
dataset = wheatDataset(df, image_dir, image_size)

In [16]:
img, dict = dataset[0]
print(dict)

{'boxes': tensor([[   0.,   69.,   37.,  130.],
        [   0.,  129.,   66.,  188.],
        [   6.,  174.,   76.,  248.],
        [   0.,  734.,   63.,  818.],
        [ 189.,  844.,  256.,  915.],
        [ 183.,  791.,  265.,  858.],
        [ 198.,  725.,  284.,  795.],
        [ 171.,  611.,  247.,  701.],
        [ 169.,  561.,  238.,  623.],
        [ 220.,  523.,  293.,  606.],
        [  45.,  357.,  165.,  478.],
        [ 344.,  314.,  435.,  397.],
        [ 330.,  660.,  408.,  725.],
        [ 327.,  777.,  406.,  850.],
        [ 331.,  721.,  404.,  776.],
        [ 428.,  705.,  519.,  766.],
        [ 404.,  756.,  476.,  857.],
        [ 489.,  751.,  568.,  839.],
        [ 555.,  754.,  625.,  837.],
        [ 581.,  701.,  690.,  750.],
        [ 626.,  625.,  714.,  698.],
        [ 596.,  795.,  682.,  908.],
        [ 640.,  747.,  707.,  814.],
        [ 291.,  861.,  350.,  928.],
        [ 744.,  789.,  835.,  862.],
        [ 835.,  679.,  925.,  765.],
  

In [20]:
# Test getting one sample
image, target = dataset[0]

# Check shapes and types
print(f"Image shape: {image.shape}")  # Should be (3, H, W)
print(f"Image dtype: {image.dtype}")  # Should be torch.float32
print(f"Boxes shape: {target['boxes'].shape}")  # Should be (N, 4)
print(f"Labels shape: {target['labels'].shape}")  # Should be (N,)
print(f"Area shape: {target['area'].shape}")  # Should be (N,)
print(f"Image value range: {image.min():.3f} to {image.max():.3f}")  # Should be [0, 1]

Image shape: torch.Size([3, 1024, 1024])
Image dtype: torch.float32
Boxes shape: torch.Size([47, 4])
Labels shape: torch.Size([47])
Area shape: torch.Size([47])
Image value range: 0.000 to 1.000


Steps:
1. Prepare data with dataset
2. Train test split
3. Dataloader setup
4. Model setup
5. train loop
6. Validation
7. Inference